In [1]:
pwd

'/home/jovyan/critical_feature_extraction/ipynb'

In [2]:
cd ..

/home/jovyan/critical_feature_extraction


In [3]:
%run __init__.py
from sklearn.decomposition import PCA, KernelPCA
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import roc_auc_score, make_scorer
from sklearn.metrics import r2_score, mean_squared_error

### Feature Extraction:
1. Check df and clean 
2. KPCA
3. Think of other non-linear dimension reduction tools.

In [21]:
apple = pd.read_csv('data/sandp500/individual_stocks_5yr_TECHNICALS_clean/AAPL_data.csv')

abbvie = pd.read_csv('data/sandp500/individual_stocks_5yr_TECHNICALS_clean/ABBV_data.csv')

cvs = pd.read_csv('data/sandp500/individual_stocks_5yr_TECHNICALS_clean/CVS_data.csv')

wfc = pd.read_csv('data/sandp500/individual_stocks_5yr_TECHNICALS_clean/WFC_data.csv')

### Clean Data:
1. Download 3 stocks' TI's
2. Check data
3. Drop unnessesary columns
4. concatenate the data

In [22]:
def clean_data(df):
    if 'Unnamed: 0' in df.columns:
        df = df.drop('Unnamed: 0', axis=1)
    else:
        pass
    if 'Unnamed: 0.1' in df.columns:    
        df = df.drop('Unnamed: 0.1', axis=1)
    else:
        pass
    if 'Unnamed: 0.1.1' in df.columns:
        df = df.drop('Unnamed: 0.1.1', axis=1)
    else:
        pass
    return df

In [23]:
apple = clean_data(apple)
cvs = clean_data(cvs)
wfc= clean_data(wfc)
abbvie = clean_data(abbvie)

In [25]:
multi_positions = pd.concat([wfc, cvs])
multi_positions = pd.concat([multi_positions, apple])
multi_positions = pd.concat([multi_positions, abbvie])

In [26]:
multi_positions = multi_positions.set_index(pd.to_datetime(multi_positions.Date))
multi_positions = multi_positions.drop('Date', axis=1)
multi_positions = pd.get_dummies(multi_positions)
multi_positions = multi_positions.sort_index()

In [29]:
multi_positions.shape 

(4677, 51)

### Building Model with KPCA & SVR:
1. Extract X and y values
2. train, test, split
3. Grid Search
    - build pipeline
    - build grid
    - preform gridsearch
    - fit pipe


In [32]:
#define the y's as the current day's trading signal 
y = multi_positions.Target
X = multi_positions.drop('Target', axis=1)

In [34]:
#because this is time-series data we must manually split the data 
X_train = X.iloc[0:3508, :]
y_train = y.iloc[0:3508]
X_test = X.iloc[3508:, :]
y_test = y.iloc[3508:]

In [36]:
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('kpca', KernelPCA()),
    ('svr', SVR())
])

In [37]:
kpca_params = {
    'kpca__n_components': np.arange(2, 11, 1),
    'kpca__kernel': ['rbf', 'sigmoid', 'cosine'],
    'kpca__gamma': np.linspace(.1, 1, 10),
    'svr__kernel':['sigmoid', 'rbf']
}

In [38]:
pipe_gs = GridSearchCV(pipeline, param_grid= kpca_params, cv=5, n_jobs=-1)

In [39]:
pipe_gs.fit(X_train, y_train)

KeyboardInterrupt: 

In [ ]:
y_pred = pipe_gs.predict(X_test)

In [ ]:
# The mean squared error
print("Mean squared error: %.2f"
      % mean_squared_error(y_test, y_pred))
# Explained variance score: 1 is perfect prediction
print('Explained Variance: %.2f' % r2_score(y_test, y_pred))

In [ ]:
plt.plot(y_pred)
plt.show()

In [ ]:
plt.plot(y)
plt.show()